# LDA에서 Author Topic Model

<img src="1.PNG">

<img src="2.PNG"> </br></br>
- (a) : LDA
- (b) : Author 모델 (저자의 정보)
- (c) : (a) + (b)
    - W라는 단어 분포를 추정할 때 들어가는 정보는 문서별 분포 정보 + 단어별(토픽별) 분포 정보
    - 우리가 추정하고 싶은 것은 저자별 문서가 다르고 저자별 문서 분포가 추가된 것
    - 즉, 어떤 저자로부터 쓰였는지에 대한 정보가 하나 더 들어온 것
    - 문서가 저자에 대한 정보에 반영되어서 확률 분포가 들어감

- Author Topic Model(ATM)은 무엇이냐 ?
    - 첫 번째 발전내용 : 옛날에 토픽 모델에 가장 많이 사용했던 데이터는 영문 뉴스 기사와 논문 데이터!
    - 논문 데이터는 어떤 글에서 어떤 토픽 모델링을 할 때 추가적인 정보를 사용할 수 없을까 고민하게됐음. 그중 첫번째 고민은 만약 어떤 토픽(문서)들이 있는데 저자가 다르다면 당연히 단어 분포도 다를거고, 그럼 토픽이 달라지지 않을까? => 이 가정을 개발해낸 분들이 LDA에서 발전된 ATM!
    - 간단히 말하면 **ATM = LDA + 저자정보**

* LDA에서 저자정보(metadata)가 추가된 토픽모델.
* 문서별 단어분포를 반영하던 파라미터에 저자별 문서분포 정보까지 추가된 형태.

- ATM을 어떻게 응용할 것이냐
- 카카오톡 대화 전체를 보고 토픽 몇개를 수정한 결과와 사용자별 메시지의 분포를 가정하고 추정한 것의 토픽 결과가 다르게 나오겠지
    - 발화하는 사람에 따라 메시지를 분류했으니까

In [38]:
import pandas as pd
import numpy as np
from pprint import pprint

import pickle

In [39]:
with open("data/cleaned_data.pk", "rb") as f:
    data = pickle.load(f)
    
data.reset_index(drop=True, inplace=True)
print(data.head())
print(data.info())

             Date User                Message
0  2018. 5. 3. 00  주현                엥 결국 안먹음
1  2018. 5. 3. 00  주현    너 그 야구부 엠티는 가는거야 그래서
2  2018. 5. 3. 00  현영                      안가
3  2018. 5. 3. 00  현영        근엄마한테갈거라고말해놓긴햇는디
4  2018. 5. 3. 00  현영                  보내준다허면
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22914 entries, 0 to 22913
Data columns (total 3 columns):
Date       22914 non-null object
User       22912 non-null object
Message    22914 non-null object
dtypes: object(3)
memory usage: 537.2+ KB
None


### ATM 사용을 위한 데이터 처리

In [40]:
users = set(data["User"])
users

{nan, nan, '영현 ', '주현 ', '주희 ', '지의 ', '지이 ', '현영 '}

In [41]:
# 사용자별로 데이터 묶기
authors = data.groupby('User')
pprint(authors.groups)
print(type(authors.groups))

{'영현 ': Int64Index([  655,   738,   742,   743,   746,   747,   750,   752,   756,
              757,
            ...
            22857, 22880, 22881, 22882, 22894, 22898, 22899, 22900, 22901,
            22913],
           dtype='int64', length=3372),
 '주현 ': Int64Index([    0,     1,    42,    50,    86,    98,   104,   110,   111,
              113,
            ...
            22782, 22786, 22787, 22788, 22790, 22824, 22844, 22845, 22846,
            22861],
           dtype='int64', length=1205),
 '주희 ': Int64Index([   18,    19,    20,    26,    36,    39,    43,    53,    57,
               89,
            ...
            22837, 22843, 22850, 22852, 22870, 22871, 22873, 22874, 22877,
            22886],
           dtype='int64', length=3292),
 '지의 ': Int64Index([    7,    27,    28,    31,    37,    38,    45,    47,    48,
               49,
            ...
            22865, 22869, 22883, 22884, 22887, 22888, 22892, 22903, 22905,
            22912],
           dtype='int64', le

In [43]:
# 묶인 데이터를 Int64Index(pandas의 데이터 타입)에서 list로 바꾸기

author2doc = {}

for user, index in authors.groups.items():
    author2doc[user] = list(index)
    
print(author2doc)

{'영현 ': [655, 738, 742, 743, 746, 747, 750, 752, 756, 757, 765, 766, 770, 775, 777, 779, 781, 786, 787, 817, 818, 820, 821, 822, 823, 824, 833, 834, 838, 839, 841, 843, 864, 865, 870, 871, 872, 874, 877, 878, 880, 881, 882, 883, 884, 885, 887, 891, 892, 893, 905, 907, 908, 909, 910, 913, 914, 918, 923, 924, 925, 930, 933, 937, 938, 941, 942, 952, 955, 956, 957, 960, 967, 993, 1041, 1045, 1047, 1048, 1052, 1053, 1081, 1082, 1091, 1092, 1103, 1116, 1117, 1129, 1136, 1155, 1164, 1166, 1168, 1169, 1187, 1190, 1191, 1192, 1195, 1196, 1198, 1200, 1203, 1206, 1210, 1215, 1229, 1230, 1232, 1234, 1238, 1243, 1251, 1252, 1255, 1263, 1264, 1270, 1273, 1280, 1298, 1305, 1310, 1311, 1380, 1381, 1385, 1388, 1392, 1399, 1406, 1407, 1408, 1418, 1442, 1463, 1516, 1530, 1532, 1534, 1570, 1573, 1574, 1575, 1603, 1604, 1623, 1625, 1632, 1634, 1638, 1641, 1642, 1644, 1649, 1650, 1653, 1654, 1665, 1666, 1674, 1719, 1721, 1722, 1725, 1730, 1731, 1733, 1736, 1743, 1757, 1781, 1782, 1817, 1818, 1861, 1862, 186

In [44]:
# gensim의 들어갈 데이터 만들기
# LDA 모델에서 파라미터 : corpus, dictionary, number_of_topics 등등

# 우리가 가지고 있는 "Message" 데이터 하나하나는 전부 str이고 str이 tokenized된 결과가 우리가 원하는 형태
# corpus와 dictionary를 만드는 input은 "list of list of word"
# 따라서 tokenized된 데이터가 필요해
tokenized_data = [msg.split() for msg in list(data["Message"])]
print(tokenized_data[:10])

[['엥', '결국', '안먹음'], ['너', '그', '야구부', '엠티는', '가는거야', '그래서'], ['안가'], ['근엄마한테갈거라고말해놓긴햇는디'], ['보내준다허면'], ['개이득'], ['파리타임'], ['마커롱', '먹고싶따'], ['마카롱임'], ['바보야']]


### gensim을 이용한 Author Topic Model

In [45]:
from gensim.models import AuthorTopicModel
from gensim.corpora import Dictionary, bleicorpus # bleicorpus: 저장할 때 쓰는 포맷
from gensim import corpora

import os

In [46]:
# ATM에 사용할 Dictionary 만들기
if not os.path.exists("kakao(ATM)_dict"):
    dictionary = corpora.Dictionary(tokenized_data)
    dictionary.save("kakao(ATM)_dict")

else:
    dictionary = Dictionary.load("kakao(ATM)_dict")

# ATM에 사용할 corpus 만들기
if not os.path.exists("kakao(ATM)_corpus"):
    # LDA 모델에서 사용하는 corpus는 document term matrix
    # 행이나 열(보통은 열-단어, 행-document), document 한 줄은 bow형태로 표현되어있음
    # 세로는 unique한 단어 개수, 가로는 문서 개수인 형태
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    # dictionary.doc2box(doc)대신 sklearn.CountVectorizer 써도 돼
    corpora.BleiCorpus.serialize("kakao(ATM)_corpus", corpus) # 피클로 저장하는 것과 비슷함

else :
    corpus = bleicorpus.BleiCorpus("kakao(ATM)_corpus")

In [47]:
# ATM에 들어갈 데이터 확인하기
# document term matrix의 형태가 보고싶은 것
print("Number of authors : %d" % len(authors))
print("Number of unique tokens : %d" % len(dictionary))
print("Number of documents : %d" % len(corpus))

Number of authors : 6
Number of unique tokens : 24796
Number of documents : 23009


In [48]:
# dictionary함수로 만든 사전에 있는 단어 보기
print(dictionary)
for idx in dictionary:
    print(dictionary[idx])

Dictionary(24796 unique tokens: ['결국', '안먹음', '엥', '가는거야', '그']...)
결국
안먹음
엥
가는거야
그
그래서
너
야구부
엠티는
안가
근ㅔ엄마한테갈거라고말해놓긴햇는디
보내준다허면
개이득
파리타임
마커롱
먹고싶따
마카롱임
바보야
야웃김
고딩때보내줫는데
내
내친구가
리얼
얼굴이
프라이팬
아개웃곀
그린줄
동그라미임
컴퍼스로
뭐지
반응은
고딩때야
개똑같아
지금이랑
귀엽지
얘그라
이거어때
똑같다니
야
귀여오
시부랭
이주희
작가임
에바참치쌈싸서
저얼굴이면
지금도
한입에털어넣야됨
털어넣으셈
근데
똑같음
참치쌈싸서
털어놓어야함
똑같다고
레알
개충격
시방
컴퍼스
잠깐만
레알똑같넼
시부랰
진현영대잔치
아닠
다른점
떡지고
안떡진고
앞머리가
개좋앜
앜
미틴
존나
행복하다
깜찍
왴
개욱겨
아근데
널
좋아해버렷
똑같이
생겼는데
똑같으면
쌈싸목는다
에바
쓸거얌
으로
저거
진짜
레알똑같을줄은
아니
자가
겟아웃오브히얼
이거
잘못하더거
단톡에
데사
보낼뻔
보내지
아
죽을뻔
죽일뻔
보내면
진현영
자퇴할까봐
휴학신청서
끄적
야주희
몰겠눙데
물어봐도되나
이런거
걍
물오봄
사귀제
정우철이랑
헐
눈치로
다아네
뭐야
나만눈치가
준내없엇군
언제부터야
준내
2주됫데
왜비밀로함
나도며칠전에앎
몰라이제
거의
공개임
애들다알어
나만몰랐쓤
나도몰랏음
야넌근데
17에도
몇명은
안댕
안다
난
말안해줙음
이걸
평생
이제물어본다고
썸인줄
젤늦게암
그리고
대답
내가할랬는데
니가말해ㅡㅡ
왜
언니그때
나
이언니
주현
화욜날
쉬는시간에
씨
웅웅
편의점에서
고백언헌더고
정우철한데
냐가
지랄할때
씨쉬는시간을쉬싸는시간으로봄
다웃었잖아
옆에
맞아
그때
말안해주고ㅠ
배창현이
그썰들었다
지의야
너무허다
눈짓으로
개웃기네
보는맛에
조용히
하라고
함
비밀로
사겼지
개사악해
미친
소리나
다웃는거
그때는
몰랐는데
긱사와소
생각하니까
뭔가
이상하길랴
웃곀
정우철카톡상메보니까
되있길래
생일축하해
사귀는가
이랬씀
그것도
그럴수잇지
보면서
좋아하니까
이럼
드라마썼구나
헌자
스뱔
1도없어
진짜눈치
도토리묵임

2
내조
병아리
뿔난
윤지
다이소갈라했는데
아ㅏ낼
존나많네
엉
녀러분내일
노트북안챙겨두되죱
블랙보드올러옴
챙기라고
어라라
고급씨
웅장문정
강의실
못바꾼대
에이씨팡
백팩하나
장만하셈
나도사고싶다능
백팩
벗
현양잔고빵원
알바비들어온다얄루
빵원
개강기념으루
얘드라
담주화요일에
롯데월드
어케생각
머임
키즈앤틴즈
교복없겠네
ㅠㅠㅠㅠㅠㅠㅠ
롯데월드메친
고딩때도안맞아서
나교복레알
졸업하자마자내다버림
그뜻으로
말한건
아닌데
또륵이다
교복
따흐흑
나둥
평상복
교복갈거면
가야해
추석이후에
뭐더라
유명한거
개슬펏는데
못타서
탈거얼마업던데
아트란티스
자이로스윙
후렌치
점검
이랑
탈거없지만
바람
환상의나라
가는거지뭐
맨날해
뭄
비안오냡
화욜
가서
기분내는고
헙
그건몰라
에버랜드가
가면
에버랜드
체력개거지되던데
상으로는
아이폰
맑대
다행스
놀이공원은
어딜가든
체력거지림
바람은
갈거면
말해야함
빨랑
빼야돼
앝
아메친현양은화욜에과외잇음
워루
워후
오늘도햇는데까먹엇네
오키
아침일찍가서
개마니하는데
돈
왜0원이냐곸
돌아오는건
수업할때
넘나촉박이냐
왙
돌았냨
넘나촉박
개꿀잼일듯
아슬아슬도착
아니먄
되자나
안먹으면
수업끝나고
월요일
4시이후
롯월
그거끊자
미련쟁이
갈지안갈지
말헤
아재믹겟다
미야옹
그만큼
움직이는대
고딩친구
바께
쟤
넘행
진현영앙가면
죄송염
헤헷둥둥이
네
나도안갈랭
ㅖ
프사랑
똑같죠
괌
말하섬
친목도모좋은지
여자같은여자는
현양은준비가덜됨
여자같은여자가아니고
원시인같은원시인임
부담같지마소
갖
인데요
먄
갖지마소
부담
부담이고뭐고
아직현양은
원시인이라안됨
개조앜
못간다구
했넹
뜨끔메라링
대충봐서
잘못봤다
갓다와
괜스레미안해지는현양
너가
가지뭐헤헷
오잉
가는거임
빠꾸없는거야
물
은듯
애
잘생겼다
이준기닮아따
뮤ㅓ대
이쁘게잘생겼다
친구임
아마
안될듯
알바하느라
개무섭다
딱무서움
저런스타일
한마디도못함
말걸엇다가
식도에펀치꽂힐거같음
생긴사람
저렇게
처음봐서
스타일인지
저귀걸이
숫자4도무서워
넌
죽엇다
요뜻
과대해석
죽어
암튼
준내무서움
나라는그릇에담기엔
너무큰아이들
아레나
엠디래
노는애네

존나꿀잼
한국영호ㅏ
미씽
소공녀
해어화
봄
개꿀잼이겟당
안녕나의소울메이트
존존존나재밌음
한국거는
볼려고
또써야됨
시로용난
토론조라
하기
아까썼지룽
바꿨다
이불
추워서
개앵간이었는데
화이트톤
갑분핑
추워
너옛날에
쓰던거네
갑분헬로키티
들두
오게
야그걸
어케들고와
부쳐야짘
택배로
잘어울리는데
헬로키티
재훈선배랑
들어달라하셈
겨울이불
들고오는거는
에바임
갑분재훈
택배부쳐야지
들어봐요
슨빼
이것쫌
으응
아닌가
제훈
재훈인가
마는
모냐
재후
제훈은
긁적
이름도
배우
재훈선뱈
물어봤자나
그거생각난다
박정욱
화냐
박정우라고
한거ㅠ
이사람이고
하자낰
제훈제훈
암
이사람
당연하지
당연하짘
저건또모야
존나마니봤는데
줴훈줴훈
화났떫
요늑힘
개콘인가
웃찾사
코빅인가
결혼했잖아
쟤네
코빅
웃찾사임몰라
결혼
다른애랑
한거다
제훈이랑
한거아님
나오던남자
안했지
줴훈이랑
결혼함
이사람랑
인스타잇는데
보고싶다
낙
이런게
인생의
갈걸그랬다
고구마전
존맛탱인데
내사랑
육전
파전
맛있겠다아
역시
쿠킹마마👍
산거야
만든거야
헐러러렁
거짓말
넷플릭스
회원인사람
가져온거임
쿡
회원인데
속았다
ㅔ
난아직집
영화관
왕십리
이거주현
이꺼다
기엽다
사야겠다
저거늠
언닌데
생곀
집인데
금방
나갈게
겨울느낌
물씬
구치ㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣㅣ
만들옷다
소떡소떡
허ㅓ
개맛있어ㅗ임
소스
개꾸덕꾸덕해보여
ㅠㅜㅠㅜㅠㅜㅠㅜㅠㅜㅠㅜㅠㅜ
떡꼬치소스
만들었쑴
고기로
보이지
맛있어보인다
고프다는
배가
점
배는고프구
졸린데
너무너무너무너무
송쨩
야오늘
잘생김
발했다
빛을
잘생김이
잘생김의
정점을
찍음
아조땐다
핸썸가이
옷이
굳임
아주
빨리가서
봐야겠다
수염의중요성
오늘옷이
공설기때
디스하려는
마음가짐이
보임
개잘생겼다고
내친구한테
말하고있었는데
우리교수님
오겡끼데스
찌찌뽕
디스하려고
옷
입고왘
사회생활선배느낌
뭔지
알것같음
바지
핑크색인거지
말하고있었음
인디핑크
정도로하자
방금너무대놓고바지열심히쳐다봤어
송xx교수님
상상안되는데
공설기
디스를
쒸발년까짘
아니무슨
나한테만해줬면좋겟어
취향이
하드하네
뭐냐곸
씨발년아
남았어
발표조
거의

그거내면
야식행사공짜인거
싶다
커피마실
커피안마시나
못마심
커피
잠못잠
안하지
갈까
보리보셈
데꼬와
보리
수업시간에
델꼬
옴
네일
다시함
옹욍
언제해보지
손톱너무
짧아졌씀
가실
크니까
낮춰서
보셈
소리
아미친
보리만
때림
저고양이
개윳기넼
진짜개웃김
또봐도
또웃김
뽀쨕하다
암것두못해ㅜㅜㅜㅜ
윤지이말이너무웃김
앜윤지의말
지금들음
아악
넣고
와랄랄
해버리고싶어
올리다가
찧었는데
캐리어
아픔
내가다아파
미췽
세수어케해
신기한게
따뜻한물에
씻으니까
풀림
피뭉친거
어우양아억
내가더아픔
으시부레
배고파죽겠씀
머좀사와
배고프댘
치킨팝
억
먹는거야
언니지금
먹고이떠
양념게장있따
밥먹을사람
개행복
kia
간담
데리러
햄찌
또사러간다고
한삼십분있다가
너무귀여워
사는거야
우리햄찌도
찍어야겠다
고슴도치키울까
고슴도치키우려면
전기장판도
사야함
온도높은데서사는애라서
따뜻하게
해줘야한데항상
도치전기장판팔더라
걍안키워
유튜브보고
고슴도치
키누고싶었는데ㅜㅜ
개겨워
키우고싶운데
토끼도
심장병있다해서ㅜㅜㅜㅜ
개귀엽다ㅠㅠㅠ
동을
존나많이쌈
토끼
아맞네
컵안에버블먹고싶어서
사러가는길
와근데
날씨개좋음
옷입기귀찮아소
나시만입고
롱퍄딩입었는데
더워디지겠씀
돌앗냐
롱패딩
개꿀잠잣음
밤에잠안와서큰일나것다
횬용이
쇼필
존나하고있는데
살라하다가도
일반인후기보고
존나빠꾸침
블랙쿠션삼
몇호샀누
13호
21호
위에부터는
밝아지는게
톤이
갈라지는거래서
로즈냐
아이보리냐
15호는
핑크베이스라소
옐로베이스로13호삼
발강지는줄
다시골라야지
알아보고
고창석닮은
발견
유튭보다가
이메일인가
수업안온다고
개빡침
쉬벌
고무잇으신분없겠지
개시러
고무없냐
데분계
모르겠디
뭔말인지
영상
왜보는지
아느사람
수업만하면
심심하다고
지루하다고
의미없다
저게더
갑
시간보내기
의미없이
더심심하누
심지어
안보임
자막도
밑에잇어서
작게보여주지
화면
러시아언지
영어인지
인투루팟투루쉬멩슁엥
안봄
봣음
5번넘게본듯
오지게틀어줌
중딩때
먼데
세얼간이
보여줄때마다
짤라서
저장면
보여줌
세얼간잌
틀어는주는데
재미없을거같아서
개꿀잼이삼
엠티가니
술집
하굑근처


아까보고
캡쳐했던거같은데
vs
쓸데없는
정성가득한
공부너무하기싫어서
청소중이다
프린터기
개꿀잼임
공부하는
흑우있나
jmt겟다
유부초밥
아니나
웹프강의자료얼마나잇나봣는데리얼
졸라많아서개깜놀
그와중에
돌릴때마다놀랍다
파파고번역기
곱슬머리로둘러싸인CSS😋
손코딩이지
컴터로시험보는거아니지
코딩나와도
컴터로
코딩할걸
번역기
잣됫네
코딩한다고
둘다함
이론이랑
말도안돼
잣같네
말한
샤브샤브머그러옴
야혀녕지의
사실몰럄
머고
그르게
남자친구염
걔가서울왓네
척이지
척하면
척하면척이지는뭐냐
안외유면
어케해
태그
한쪽되지
어디까지야
데분
넘어가
몇장까지야
결코
못넘어가겟음
6장
6장도
족같음
배운거
어디까지냐고
안들어서
배운데가
어디까지인지
강의자료올라온거
anybody
else
아는
배운데까지겟지
학교인사람
맞으러감
입술필러
컵안에버블
슬라임만지다가
아미친존나웃겨
니남친
안돼안도ㅑ
안된다니깐
조난애절하게
안들어
나도개당황해쑼
마지막에
웃음
큐ㅠㅠ
개웃기넼케
맞으러움
양치
더럽게하시넴
양치가
마취크림발라논거라고
앎
후크
뒤로돌리세염
관찰력
목걸이가
돌림
친절하시네염
입술
필러맞으면
두꺼워지는고임
두껍기도허고
모양이
예뻐짐
이사람이
이롷세
오동통통
Up
Volume
함정있는데요
3일간
부어있어서
끼구
댕김
입하ㅏ
탱글탱글하다
물어뜯고싶다
77ㅑ
입술조심해야댈듯
조만간
터지게따
찡떡개존맛탱
너오늘
노네
알차게
풀코스로
찐탱이네
55
군대가기전
마지막이야
갠톡이랑
단톡
답장잘못한거
갠톡인줄알고
아님아님
종맛턍
어디누
그리네임또
그리네에
앞에
존잘남있씀
후후
번따
따냐
존잘이라구
봐야됨
인지궁금
잘생겼다는사람
몰래
찍어줄까
범죄임
존잘남
개부어있는데
범죄
에바쓰
잘생겨서
저남자
쳐다봤는데
힐끗
의식하는듯
고만봐
그만보고싶은데
글로감
그만봐
씨댕
집간다
왜가는데
눈치채게하면
흘끗보는거
남자들은
으쓱거린다고
잘난줄알고
안그래도
알껄
지잘난거
거울봐서
공부하러와서
책펴놓고
폰하고
노트북만
존나하다가
그런겨
개도랏다
양
어디하구있써
웹프해
데분개는
하구
학
되나요
공부도
해야하는데
낼까지
동고강

...

됬다
싶었쥬
8시에일어남
나도근데
꺄웠는데
멕컵까지
엔나
워쉬
지렸쥬
8시50분이라
래서
뭐라고오
출튀할걸
방금와서못함
ju
run
출튀하고싶다
앉을걸
우리기억할거같음
여기앉아서
숟가락얹기
꼽준거임
떠들어서
올린지
꼽이었음
멋쓱
끝나면
과방임
밥먹자
곡
님
액
드라큘라
와꾸왜저럼
같누
퉁퉁이
어디람
술마셨누
알바하는와중에도
갬성샷
와중에도
배열시켜놓고
사장님이
안쳐다밨을까
도와주는가임
사장님
업로드영
인스타를
홍보용으로
단축키임
쥐기네의
아무리봐도
인스타있끔
내려감
기달려
올라가
올라가서
할일이잇어
4625
역전알머니
갈분
구람
간다면
주릐
가줌
왕십리더
당첨
영련
역전할머님
오빠랑간다매
알바야
군대에서
안나와서
못기다림
모든
벗어버림
허물을
짱구되보림
오일간
짱구가
안진한데
개잘됬는데
짱구의
짱도아님
개진래
실제로
개진래씨
연래지는거임
연래씨
진래얀랰1
야시발
과제뭐냐
수업pdf제출이
아그거
안내도됨
말앗음
과제라돈데
메
받아봐
소식인걸
알바중
등장하라
담주수업시간에
이어서
한다매
자정까지라는데
설명쫌
아예뭔지
올려져있음
과제때메
저나함
9주차에
올려져있땅
안내도된다는데
푸는거임
오늘까지면
제출란이
해야겟2지
올린시간이
올리고
말바꾼거아니냐
마감시간이
이거네
저렇게올리고
말바꿈
유후
마감일은
유휴인
114
말바꾼거
월욜이냐ㅡㅡ
다음시간에
이어서하자고
했쟈나
해줘야지
개꿀메라
조아여
이러곸
이럭
다행이네
팽팽놀고있었는뎈
걍해야지
시작한김에
혼자못해
덥어
봐야징
띵작으로
넷플뤽스
안봐서
이년아
저번에추천해줬잖아
유투브
재밌는
추천해죠
고시원
잘갱긴
댁글
도련님이래
유튭은
댓글이
먹여살린단
그거과제
담주까지라캤음
술마실분
아무도없었다•••
다들오ㅑ캐빨리일어난거야
나도12시
새나라의
얼은이세요
찐탱
배고츤데
뭐먹지
드쇼
팍마
잤는뎈
귀엽
돼지임
재
사람이니햇던애
일어난
젖됚꺼
다시자서
끄지
생공을
이래
곧가니까
시바뭐얔
받자
주릐도감
내이름
갈둣
입찢고깊었어
배고차
홀에번주토요일
얘네
오늘말하는거임
연습이가
지이도
말한줄
지이언니가
내얼굴
죤나무섭네
옆모습
준예
분

보는것이
오빠친구들한테
너보여주는데
3초보여주면
싸대기
갈겨
중요하지
모가부끄러운데
m성향이니
어쩌니
얘기할땐언제고
됬냐
안부꾸럽냐고
괜찮음
잘나온거임
Ulike임
사줘라
안경하나
괜찮으신데왜
안경만
뿔테밖에
안보이긴해
을듯
시력
아니저거
개나빠서
안경
얇은거
눈작아지는
안경임
존나못생긴줄
못바꿈
데를
못삼
보는눈없어서
안경사서
줄때
줘
애매
찐따니까
이럴수도
뿔테말고도
어울릴것같은데
이런안경들도
한번써봐
라고해
지의ㅣ기분나쁠수도
고맙다다들
골라줘
디자인을
끝에
문장
쓸때
되x돼o임
알고있었으면
맨마지막엔
돼임
돼지
말하고싶엇는데
사실나도
왜말안해줬냐
할까봐
되
말고는
문장끝
돼는
넣어봐
붙여보면됨
안하
이거는말이안되자나
헷갈리지말자
비주얼에서
들여쓰기
여러줄
한번에땡기는법
드래그해서
하면안됑
탭하는거말고
없애는거
구글에게물어봐
누가ㅏ알려줬는데ㅜ
이거옛날에
어쩃든
안경바꿔주고
자리한번
시러
바꿔주고
rmrj
자동교정하는건
다ㅏㅏ맞춰주는거
rmrj는
아렘알쥐
또말만드네
컨트롤kd
외워두셈
자동정렬
알트f8
자동정렬임
알트에프팔
안됨ㅠ
아무튼
현녕
자리만들어
컨트롤
로외우셈
dk
개똒똒하네
쿠
dfs까지만
했으면
그거볼때마다
아무것도모르겠고
BTS
bfs도
아이유임슬옹
잔소리
나이거
살고임
이런걸
임명해드림
주접킹
어따쓰게
쓰고와
남친생겼어
개오랜만이다
대학우째됬어
다넣어서
특교과로
그중에
하나로
발표났어
마이컸네
사귀고ㅠㅠㅠㅠ
노는중
와ㅏ주현
살았어
폰없이
가나보네
주혀니ㅣㅣ
개시끄러웠을텐데
카톡안깔움
깔아따
얼마전에
잘지내지
어땨
과제얘기하니까
모리아프네
과제가
그리워
골아프네
심심해
과제중이여
강적
겨수님만나서
대박이야
좆같은
새로오셨어
알바해
무ㅏ야
우리배움도대체
서빙
우리땐
타이타닉
에ㅔ무슨과목이야
저거보니까
공설기했던거
개어랜만
꼬마아가씨
미니벨••
꼬마공주라
저장했는데
재생해봐
카메라워킹에놀랄껄
개좋네
화질
메리
크리스마스
허호호
데이투
한구임
저건그냥
베이직하우스
이분이랑
구경
띄우냐고
수면위로
턱쪽이
어맞아
오빠도약간
턱주가리임
하관쪽이
예리하지


구냥구로타굼
욕할듯
입자하면
영현쓰
리처드슨
보외법부터
리처드근
보외법
난이미버림
리퍼드슨그게뭔지도모룸
안들어감
중간이후에
안들어가는걸루
알고잇엇름
확실은
낼물어바
괜찮은거
비슷해서
선대랑
어쩌구보단
낫겟지
점마는
1도없음
체계적인게
물어봐야
시켜노ㅗ고
플젝이든뭘든
보내고싶다
당황좆같음을
한번에표현한
충전시켜주고싶음
내폰도
나3퍼임
수업인데
진도나가버리네
문득
개빡세게하고오고
그뒤로는
쌩얼로오는듯
이제일어났다
같노
나보는거
수업감
왔지
녹음쫌
피곤해죽을꺼같음
웅우에
킴
맨뒷자리라서
안들릴수도
아결석
마지막날까지
수업해야되나
내말그ㅏ
내말그말
ㅠㅠ시발
니얘기
안궁금해
어쩌라고
추움
주연하다•••
안추움
안괴롭힌데
우리를
지입으로
매주과제냈잖아
초반에
괴로워
존재자체가
미친생색
안내는듯
입터지냐고
매주냈잖아
수업끝내라고
이번학기
피해라
휴학하고나서
유주연잇으면
짤리면
짤려
재때메
휴학할까
개에버야
한번은
휴학
우리과꺼들어야하는데
전공수업있으면
아설마
이학년에
머먹을까
밖으로
나가세
암껏두
야식행사먹고
살빠지겠다
글켔지
학교밖
렉
뒤짐
밤에잠이
누워서
어제도
잠안와서
폰하다가
어떻게그러지
야식안먹어서
영화소개영상만
유튜브로
어제도4시에쳐잠
12시만되면
시계임
바른생활의
표본
5시반에잠
웹툰보다가
시간개뚝딱
폰쫌
양념게장급
해뜰라캄
닭갈비먹고싶다
공부쫌
할라카면
해지고
알람맞추는데
우애야되노
우야노
말투라고
들린거
흫
소리니
안들려
맞는거같는데예
complex
matrix
기절할뻔
점심머거거
머거거아니고
머머거
인생얘기
뭐니
선대니
존나웃가다
석관동
어떠삼
about
how
that
안땡기지
아니묜
그럼그냥
커피나무
긔
무얮
무야지
땡겨
당연한소리를
최소1개이상
학식만은
그럴바에
섞관동
먹을게
아니미욘
머먹고싶엉
점심먹으러
안오자나
학굥거야
먹고싶츤걸
얘기한거구나
엽떡먹잖아
중당중당
개오래걸리듯
맛잇겟다
4끼째
먹는즁
육개장만
질린다
글애
erp만
간다구용
어케나눠
쪼그만거할래
맨뒤에꺼
온줄
욕하길래
주얀이
욕하기
암거나
기사
읽어도됨
읽으면되지않나
234쪽할게
전해

훨신쉽데
할거삼
키우고픔
나쟤알아
강아지랑보드카페
거기잇는
고양아냐
헉니유학간다고
준비를
해본다구
굠둥
강쥐보단
고냥이키우고싶음
보리제외
나도쟤있음
이쁘제
저거뭔데
산거임
양초
현영하나만
골르라해서
봄제주고름
제주도인
선물받는겨
봄제주랑
월정리가
젤이쁘다
에메뢀드
4가
예뿐데
다지나갔네
카톡이제봄
핫플
강의녹화했는데
깊은좌절을느끼며
겨수님들
돈받고
할일
동영상올리기
얼마나싫었을까
싫었는데
알았을때도
프로젝트올려야하는거
깊은좌절
강의도
올려하잖
쥰비해서
좌절감
집배경
다나온데
갸웃김
고민상담
들어줄사람
없어도
일단말하고보겠음
어서읽어라
Hurry
her
we
가져야지
뭘안되
시간을
안되긴
일주일동안연락을안하면
미치겠어서
식음
오빠헌데
그런거같기도하고
그동안
뭐했난지
믿음이
시간갖자고
연락안하면
의심가고
못만날껄
처음이랑
만나보다가
맘떨어질때
헤어지셈
근데헤어지고
그리워하면서
울면
진짜맘떨어질때가언제일까•••
기준을몰겠삼
연애
아직은
질껑
갤러리
뒤적거릴거같은데
때문에
불가능일껄
있기
잘지내는건
정이
헤어져도
홀가분하게
이때
재밌었지
그러니깐
내가지금
사랑해서
정으로만나는건지
만나는건지
구분이안됨
고민도
헤어져야될때는
헤어지고
여즘
짜증남
잘못해서
혼나도
내가혼나는게
짜증나고
진심으로
풀어주고싶지가않음
너무많이혼나ㅓ
혼나서
그런거아님
삐지고
화내는것도잇는듯
갑니다
공감
누르고
고랭자주
그런걸수도
싸워서
내가잘못했는데도
싸우는게지쳐벌읨
그런거임
맘이
맘이뜨먄
꼴보기
사과하기
싫고
잘못해도
져주기
사과하기싫고
원래도
내문제인건가
찐이당
귀찮음이란
나일거야
부정할수없다
뭐때매
화내길래
글겡
뭐잘못했오
오빠가화난부분은
남자친구랑
비교하는말을했삼
자기집에데려다달라하고
지윤이남자친구는
집데려다주는데누구는
개화냄
오빠개빡쳐서
안붙잡아서
더화냄
장난스럽게했으면
저말을
그렇게말할수도잇지
안붙잡음
비교하는거
잘못이긴한데
안델따주고
왜집에
니보고
델따달라함
영화봤는데
바로앞에서
지하철탈수있었믄데
15분정도
걸어가야있거던
오빠집
오빠집까지데려다줄래
이래가지고
안잡고
지하철타고
지쳤으면

1라도
남아있으묜
양심이
상업적잉라서
정떨어짐
뱉어내
위로금이라도
상업적으로
생각하는거밖에
학생을
위한
학교시설도
학굔데
학생보단
돈버는
용이고
외부인용이고
상업단지를
이럴거면
지어야지
학교말고
에스컬레이터가동
외부인올때만
헤어지고왓어
뻥찼지
란잔하자
다음주에
썰은
술마시면서
로주
롼잔하자
로주롼잔은누가알려줬니
ㅠㅠ글쿠나
가구싶네잉
영현이이이ㅠㅠ
잘했어
울컥스여
잘했다는
날잡고
놀러왕
언제한번
끝나면은
모르겠지만
볼지
코다차야가
민수가
쥰나많이갔다는디
압구정이래
지점여러개임
근데ㅔ그때
술안먹는애들
있지않나
자구시험이야
칭긔들
근데뭐
가고싶은대로가면되지모
놀애들
에바당
멀리가면
오메근데
시험은
오프라인으로봄
그럴라했는데
바꼈으면
시험도
좋겠당헤헷
과제대체했으묜
에이가자
절대평가다같이
웨않자
깨있는
시간아냐
이시간은
고롬
서식시간인걸
새삼
웨않자냐니
당여나지
하위다들
밤이되었습니다
고개를
들어주세요
마피아들은
죽일게요
스륵••
마피아야
두명이었던거임
마피아
6명인데ㅜ마피아가
둘이라••
쉽구만
공분는
잘하고있나유
시험기간인게
그래써
학교다닐땨도
근데ㅔ싸강으로
대학수업들으면
시험안칠확률이
높다고하니까
궁금하다
엌던지
더실감안남
같고
듣기싫어
개조은디
아니근학교안가서
우주공강두없구
죳망했는데
미룰수있어서
좋긴해
장점이라면
딱인데
기분도
휴학한
나중은
과제라
그래성
설문조사
해주랑
해쏘요
앉았는
임산부삭
나두했삼
고마워❤
맛집
있찌
열불날개
먹어봄
날개랑
닭발은
막창
였던거같움
막창사랑해
마싯음
막창이
왜ㅐ너네만가
암되겟다
헤쳐모여
월요일이나
바쁘지ㅣ
놀아줄
없지ㅣ
아루바이투
일욜이면
고고
다닌당
미술학원
너근데좀잘그림
꿈꾸며•••
전과를
좋겠당
오일파스텔
개부러
유화
Wow
미완성인데
요오
눈썹같은거
밑색칠한고
크게크게
그렸네👍👍
잘한다
달리밴퓌카소
등쟝
그려줘라
유명해지기전에
그럴일없어
일주일기다림
재밌어서
취미미술인데
뿌듯할듯뿌듯뿌듯
완성되면
개잘했다
와멋지다
미밴드
3만원대
기능
그게뭐죠
방수
애플워치
다할수있으몀
애플워치급
3만원이야
다되는데
샤오미라
싸당
없는

In [49]:
for cp in corpus:
    print(cp) #(단어, 빈도)

[(0, 1.0), (1, 1.0), (2, 1.0)]
[(3, 1.0), (4, 1.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0)]
[(9, 1.0)]
[(10, 1.0)]
[(11, 1.0)]
[(12, 1.0)]
[(13, 1.0)]
[(14, 1.0), (15, 1.0)]
[(16, 1.0)]
[(17, 1.0)]
[(18, 1.0)]
[(19, 1.0), (20, 1.0), (21, 1.0)]
[(22, 1.0)]
[(23, 1.0)]
[(24, 1.0)]
[(25, 1.0)]
[(26, 1.0), (27, 1.0), (28, 1.0)]
[(4, 1.0), (29, 1.0), (30, 1.0)]
[(31, 1.0)]
[(32, 1.0), (33, 1.0)]
[(34, 1.0), (35, 1.0), (36, 1.0)]
[(37, 1.0), (38, 1.0)]
[(39, 1.0)]
[(40, 1.0)]
[(41, 1.0), (42, 1.0)]
[(43, 1.0), (44, 1.0), (45, 1.0), (46, 1.0)]
[(47, 1.0)]
[(48, 1.0), (49, 1.0)]
[(50, 1.0), (51, 1.0)]
[(52, 1.0), (53, 1.0)]
[(54, 1.0), (55, 1.0)]
[(56, 1.0)]
[(57, 1.0)]
[(58, 1.0)]
[(59, 1.0)]
[(60, 1.0)]
[(61, 1.0)]
[(62, 1.0)]
[(63, 1.0), (64, 1.0), (65, 1.0)]
[(66, 1.0)]
[(67, 1.0)]
[(68, 1.0)]
[(69, 1.0), (70, 1.0)]
[(71, 1.0)]
[(72, 1.0)]
[(73, 1.0), (74, 1.0)]
[(75, 1.0), (76, 1.0)]
[(77, 1.0), (78, 1.0)]
[(79, 1.0)]
[(80, 1.0), (81, 1.0)]
[(82, 1.0), (83, 1.0), (84, 1.0), (85, 1.0)]
[(8

[(2209, 1.0)]
[(2210, 1.0)]
[(5, 1.0), (616, 1.0), (2211, 1.0), (2212, 1.0)]
[(2213, 1.0)]
[(2214, 1.0)]
[(2215, 1.0)]
[(2216, 1.0)]
[(6, 1.0), (2217, 1.0)]
[(87, 1.0)]
[(2218, 1.0)]
[(2219, 1.0)]
[(2220, 1.0), (2221, 1.0)]
[(2222, 1.0), (2223, 1.0)]
[(971, 1.0), (2224, 1.0)]
[(616, 1.0), (2225, 1.0)]
[(2226, 1.0)]
[(85, 1.0)]
[(2227, 1.0)]
[(227, 1.0), (1918, 1.0), (2228, 1.0)]
[(2229, 1.0), (2230, 1.0)]
[(233, 1.0), (2231, 1.0), (2232, 1.0)]
[(463, 1.0), (656, 1.0), (2233, 1.0)]
[(463, 1.0), (656, 1.0), (1097, 1.0)]
[(463, 1.0), (2234, 1.0)]
[(2235, 1.0), (2236, 1.0), (2237, 1.0), (2238, 1.0)]
[(2239, 1.0)]
[(607, 1.0), (2240, 1.0)]
[(2241, 1.0), (2242, 1.0)]
[(2243, 1.0)]
[(529, 1.0), (2244, 1.0)]
[(2245, 1.0), (2246, 1.0)]
[(259, 1.0), (2247, 1.0), (2248, 1.0)]
[(1179, 1.0), (2249, 1.0), (2250, 1.0)]
[(2240, 1.0), (2251, 1.0), (2252, 1.0)]
[(2253, 1.0)]
[(85, 1.0), (2254, 1.0)]
[(48, 1.0), (2255, 1.0)]
[(2256, 1.0)]
[(853, 1.0), (2257, 1.0)]
[(2258, 1.0), (2259, 1.0)]
[(2260, 1.0)]

[(2862, 1.0)]
[(1284, 1.0)]
[(2489, 1.0)]
[(90, 1.0), (3902, 1.0), (4616, 1.0)]
[(608, 1.0), (609, 1.0)]
[(4617, 1.0), (4618, 1.0)]
[(4619, 1.0)]
[(407, 1.0), (3256, 1.0), (4620, 1.0)]
[(494, 1.0), (608, 1.0), (609, 1.0)]
[(84, 1.0), (147, 1.0), (4621, 1.0)]
[(4622, 1.0), (4623, 1.0)]
[(85, 1.0)]
[(4624, 1.0)]
[(134, 1.0), (648, 1.0)]
[(247, 1.0), (4625, 1.0), (4626, 1.0)]
[(4627, 1.0), (4628, 1.0), (4629, 1.0)]
[(259, 1.0)]
[(84, 1.0), (4630, 1.0)]
[(48, 1.0), (4631, 1.0), (4632, 1.0)]
[(4633, 1.0), (4634, 1.0), (4635, 1.0), (4636, 1.0)]
[(108, 1.0), (4637, 1.0)]
[(4638, 1.0)]
[(2848, 1.0), (4639, 1.0), (4640, 1.0), (4641, 1.0), (4642, 1.0)]
[(4618, 1.0), (4643, 1.0), (4644, 1.0)]
[(4645, 1.0)]
[(85, 1.0), (111, 1.0), (4646, 1.0), (4647, 1.0), (4648, 1.0)]
[(147, 1.0), (608, 1.0), (4649, 1.0)]
[(2939, 1.0)]
[(2294, 1.0), (4650, 1.0), (4651, 1.0)]
[(4652, 1.0), (4653, 1.0), (4654, 1.0), (4655, 1.0), (4656, 1.0)]
[(429, 1.0), (1040, 1.0), (4657, 1.0)]
[(4658, 1.0)]
[(1752, 1.0)]
[(90, 1

[(145, 1.0), (268, 1.0), (6756, 1.0), (6888, 1.0), (6889, 1.0)]
[(2843, 1.0)]
[(6890, 1.0)]
[(96, 1.0), (6891, 1.0)]
[(48, 1.0), (90, 1.0), (6892, 1.0)]
[(6893, 1.0)]
[(6894, 1.0)]
[(6895, 1.0)]
[(96, 1.0), (134, 1.0), (6896, 1.0), (6897, 1.0)]
[(96, 1.0), (3295, 1.0), (6898, 1.0), (6899, 1.0), (6900, 1.0)]
[(268, 1.0), (1288, 1.0), (6901, 1.0)]
[(6902, 1.0), (6903, 1.0)]
[(6904, 1.0), (6905, 1.0), (6906, 1.0)]
[(48, 1.0), (108, 1.0), (6907, 1.0)]
[(6908, 1.0), (6909, 1.0), (6910, 1.0), (6911, 1.0)]
[(793, 1.0), (6912, 1.0), (6913, 1.0), (6914, 1.0)]
[(5249, 1.0), (6915, 1.0), (6916, 1.0)]
[(5, 1.0), (6917, 1.0), (6918, 1.0), (6919, 1.0)]
[(96, 1.0), (6920, 1.0)]
[(153, 1.0), (6921, 1.0)]
[(2689, 1.0), (6922, 1.0), (6923, 1.0), (6924, 1.0)]
[(6925, 1.0)]
[(6926, 1.0)]
[(48, 1.0), (3315, 1.0)]
[(989, 1.0), (6927, 1.0), (6928, 1.0)]
[(2843, 1.0)]
[(429, 1.0), (6929, 1.0)]
[(6930, 1.0)]
[(6931, 1.0), (6932, 1.0)]
[(5, 1.0), (6933, 1.0)]
[(607, 1.0)]
[(147, 1.0), (6934, 1.0)]
[(410, 1.0), 

[(9091, 1.0)]
[(341, 1.0), (9092, 1.0)]
[(9093, 1.0)]
[(9093, 1.0), (9094, 1.0), (9095, 1.0)]
[(96, 1.0), (9096, 1.0)]
[(3136, 1.0), (9097, 1.0), (9098, 1.0)]
[(9099, 1.0)]
[(6852, 1.0), (9100, 1.0)]
[(9101, 1.0)]
[(6286, 1.0)]
[(9102, 1.0)]
[(802, 1.0), (9103, 1.0)]
[(112, 1.0)]
[(9104, 1.0), (9105, 1.0)]
[(2238, 1.0)]
[(1077, 1.0), (3150, 1.0), (9106, 1.0)]
[(2502, 1.0), (5287, 1.0)]
[(147, 1.0), (552, 1.0), (9107, 1.0), (9108, 1.0), (9109, 1.0)]
[(9110, 1.0)]
[(9111, 1.0)]
[(909, 1.0)]
[(9112, 1.0)]
[(112, 1.0)]
[(8631, 1.0)]
[(9113, 1.0)]
[(9114, 1.0)]
[(9110, 1.0), (9115, 1.0), (9116, 1.0), (9117, 1.0), (9118, 1.0), (9119, 1.0), (9120, 1.0)]
[(85, 1.0), (9121, 1.0), (9122, 1.0)]
[(9123, 1.0)]
[(170, 1.0)]
[(261, 1.0), (9124, 1.0)]
[(429, 1.0), (9125, 1.0), (9126, 1.0)]
[(220, 1.0), (9127, 1.0)]
[(9128, 1.0)]
[(87, 1.0), (554, 1.0), (6062, 1.0), (9129, 1.0), (9130, 1.0)]
[(853, 1.0), (9093, 1.0), (9131, 1.0), (9132, 1.0), (9133, 1.0)]
[(366, 1.0), (9134, 1.0)]
[(115, 1.0)]
[(320, 1

...

[(691, 1.0), (9261, 1.0), (10082, 1.0), (12402, 1.0), (12562, 1.0), (16612, 1.0), (16616, 1.0), (16617, 1.0), (16618, 1.0), (16619, 1.0), (16620, 1.0)]
[(318, 1.0), (16621, 1.0), (16622, 1.0)]
[(112, 1.0), (115, 1.0), (2294, 1.0), (16520, 1.0), (16623, 1.0), (16624, 1.0), (16625, 1.0), (16626, 2.0), (16627, 1.0)]
[(38, 1.0), (77, 1.0), (784, 1.0), (1042, 1.0), (2294, 1.0), (16623, 1.0), (16624, 1.0), (16625, 2.0), (16626, 1.0), (16628, 1.0), (16629, 1.0), (16630, 1.0), (16631, 1.0)]
[(16632, 1.0)]
[(147, 1.0), (11936, 1.0), (14723, 1.0), (15979, 1.0), (16633, 1.0), (16634, 1.0), (16635, 1.0)]
[(2, 1.0), (16630, 1.0), (16636, 1.0)]
[(147, 1.0), (3902, 1.0), (16637, 1.0), (16638, 1.0), (16639, 1.0)]
[(16640, 1.0)]
[(429, 1.0), (784, 1.0), (13597, 1.0), (16641, 1.0), (16642, 1.0)]
[(16643, 1.0), (16644, 1.0)]
[(16645, 1.0)]
[(2473, 1.0), (16646, 1.0)]
[(3714, 1.0), (16647, 1.0), (16648, 1.0)]
[(16649, 1.0)]
[(147, 1.0)]
[(16650, 1.0), (16651, 1.0)]
[(917, 1.0)]
[(147, 1.0), (1229, 1.0)]
[

[(18638, 1.0)]
[(147, 1.0)]
[(18639, 1.0)]
[(18640, 1.0)]
[(145, 1.0)]
[(2611, 1.0), (18641, 1.0)]
[(18642, 1.0), (18643, 1.0)]
[(9408, 1.0), (18644, 1.0)]
[]
[(38, 1.0)]
[(48, 1.0), (18645, 1.0)]
[(18646, 1.0)]
[(9773, 1.0)]
[(18647, 1.0), (18648, 1.0)]
[(18649, 1.0)]
[(18650, 1.0)]
[(289, 1.0), (8722, 1.0), (14916, 1.0), (15046, 1.0), (18651, 1.0), (18652, 1.0), (18653, 1.0), (18654, 1.0), (18655, 1.0)]
[(264, 1.0), (308, 1.0)]
[(18656, 1.0)]
[(18657, 1.0)]
[(18658, 1.0)]
[(595, 1.0), (18176, 1.0), (18659, 1.0), (18660, 1.0)]
[(18661, 1.0), (18662, 1.0)]
[(3997, 1.0), (18663, 1.0)]
[(18664, 1.0), (18665, 1.0), (18666, 1.0)]
[(18667, 1.0)]
[(96, 1.0)]
[(18668, 1.0)]
[(11674, 1.0), (18669, 1.0)]
[(4851, 1.0)]
[(18670, 1.0)]
[(18671, 1.0)]
[(18672, 1.0)]
[(233, 1.0), (18673, 1.0)]
[(13917, 1.0), (18674, 1.0)]
[(18649, 1.0), (18675, 1.0)]
[(868, 1.0), (18676, 1.0)]
[(18677, 1.0)]
[(18678, 1.0)]
[(555, 1.0)]
[(147, 1.0), (4651, 1.0), (18679, 1.0), (18680, 1.0)]
[(147, 1.0), (1042, 1.0), (

[(85, 1.0), (20291, 1.0), (20292, 1.0)]
[(20293, 1.0), (20294, 1.0), (20295, 1.0)]
[(585, 1.0), (10229, 1.0), (20296, 1.0)]
[(18448, 1.0)]
[(87, 1.0)]
[(20297, 1.0), (20298, 1.0)]
[(20299, 1.0)]
[(3463, 1.0), (20300, 1.0)]
[(680, 1.0), (20301, 1.0)]
[(69, 1.0), (20302, 1.0)]
[(20303, 1.0)]
[(20304, 1.0)]
[(6076, 1.0), (7921, 1.0)]
[(6444, 1.0), (8723, 1.0), (14399, 1.0)]
[(289, 1.0)]
[(96, 1.0)]
[(18685, 1.0), (20305, 1.0)]
[(20306, 1.0)]
[(20307, 1.0), (20308, 1.0)]
[(20309, 1.0), (20310, 1.0)]
[(17046, 1.0), (20311, 1.0)]
[(20312, 1.0), (20313, 1.0)]
[(147, 1.0), (3772, 1.0), (20314, 1.0)]
[(470, 1.0)]
[(20315, 1.0)]
[(5, 1.0), (20316, 1.0), (20317, 1.0)]
[(20318, 1.0), (20319, 1.0), (20320, 1.0), (20321, 1.0)]
[(20322, 1.0), (20323, 1.0)]
[(7373, 1.0)]
[(19419, 1.0)]
[(12389, 1.0), (20324, 1.0)]
[(305, 1.0), (20325, 1.0), (20326, 1.0)]
[(20327, 1.0)]
[(20328, 1.0)]
[(20329, 1.0)]
[(961, 1.0), (20330, 1.0)]
[(6422, 1.0), (8988, 1.0), (20331, 1.0), (20332, 1.0)]
[(20333, 1.0), (20334,

[(340, 1.0), (8022, 1.0), (21814, 1.0)]
[(8362, 1.0)]
[(21815, 1.0)]
[(2611, 1.0)]
[(10422, 1.0)]
[(21816, 1.0)]
[(153, 1.0)]
[(21817, 1.0)]
[(112, 1.0), (21818, 1.0)]
[(410, 1.0)]
[(21819, 1.0), (21820, 1.0)]
[(472, 1.0), (6650, 1.0)]
[(21821, 1.0)]
[(233, 1.0), (774, 1.0), (21822, 1.0)]
[(7376, 1.0), (21823, 1.0), (21824, 1.0)]
[(270, 1.0)]
[(21825, 1.0)]
[(308, 1.0), (8362, 1.0), (21826, 1.0), (21827, 1.0), (21828, 1.0)]
[(2517, 1.0)]
[(1744, 1.0)]
[(21829, 1.0)]
[(21830, 1.0)]
[(38, 1.0), (21831, 1.0)]
[(21832, 1.0)]
[(4808, 1.0)]
[(21833, 1.0)]
[(12366, 1.0)]
[(21834, 1.0)]
[(1774, 1.0)]
[(21835, 1.0)]
[(318, 1.0)]
[(21836, 1.0)]
[(21837, 1.0)]
[(21838, 1.0)]
[(289, 1.0), (21839, 1.0)]
[(410, 1.0), (429, 1.0), (15309, 1.0)]
[(185, 1.0)]
[(21840, 1.0)]
[(18409, 1.0)]
[(616, 1.0), (6114, 1.0), (21841, 1.0)]
[(96, 1.0), (21842, 1.0)]
[(7181, 1.0), (21843, 1.0)]
[(1620, 1.0), (21844, 1.0)]
[(521, 1.0), (8240, 1.0)]
[(1771, 1.0), (21845, 1.0)]
[(21846, 1.0)]
[(14338, 1.0)]
[(10686, 1.0

[(23310, 1.0)]
[(21726, 1.0), (23311, 1.0), (23312, 1.0), (23313, 1.0), (23314, 1.0)]
[(23315, 1.0)]
[(912, 1.0)]
[(472, 1.0), (23316, 1.0)]
[(23317, 1.0), (23318, 1.0)]
[(264, 1.0), (23319, 1.0)]
[(23320, 1.0)]
[(23321, 1.0)]
[(23322, 1.0)]
[(8177, 1.0), (10621, 1.0), (23323, 1.0)]
[(1742, 1.0), (7789, 1.0), (23324, 1.0)]
[(1378, 1.0), (21887, 1.0)]
[(850, 1.0), (23325, 1.0)]
[(8628, 1.0), (23326, 1.0)]
[(268, 1.0), (6288, 1.0), (23327, 1.0)]
[(96, 1.0), (8628, 1.0), (18629, 1.0), (23328, 1.0)]
[(407, 1.0), (7847, 1.0), (23329, 1.0)]
[(5309, 1.0)]
[(9928, 1.0)]
[(23330, 1.0)]
[(3298, 1.0), (23331, 1.0)]
[(472, 1.0), (23332, 1.0)]
[(527, 1.0), (853, 1.0), (23333, 1.0)]
[(527, 1.0)]
[(23334, 1.0)]
[(23335, 1.0)]
[(2925, 1.0)]
[(16697, 1.0)]
[(23336, 1.0)]
[(96, 1.0), (3090, 1.0)]
[(147, 1.0), (1067, 1.0), (23337, 1.0), (23338, 1.0)]
[(6244, 1.0), (23339, 1.0)]
[(23340, 1.0)]
[(527, 1.0), (23341, 1.0)]
[(3581, 1.0), (23342, 1.0)]
[(23343, 1.0), (23344, 1.0), (23345, 1.0)]
[(9928, 1.0)]
[

In [50]:
# 사람이 이해할 수 있는 형태로 코퍼스 사전 재구성 해보기 (term-frequency)
[[(dictionary[id], freq) for id, freq in cp] for cp in corpus ]

[[('결국', 1.0), ('안먹음', 1.0), ('엥', 1.0)],
 [('가는거야', 1.0),
  ('그', 1.0),
  ('그래서', 1.0),
  ('너', 1.0),
  ('야구부', 1.0),
  ('엠티는', 1.0)],
 [('안가', 1.0)],
 [('근ㅔ엄마한테갈거라고말해놓긴햇는디', 1.0)],
 [('보내준다허면', 1.0)],
 [('개이득', 1.0)],
 [('파리타임', 1.0)],
 [('마커롱', 1.0), ('먹고싶따', 1.0)],
 [('마카롱임', 1.0)],
 [('바보야', 1.0)],
 [('야웃김', 1.0)],
 [('고딩때보내줫는데', 1.0), ('내', 1.0), ('내친구가', 1.0)],
 [('리얼', 1.0)],
 [('얼굴이', 1.0)],
 [('프라이팬', 1.0)],
 [('아개웃곀', 1.0)],
 [('그린줄', 1.0), ('동그라미임', 1.0), ('컴퍼스로', 1.0)],
 [('그', 1.0), ('뭐지', 1.0), ('반응은', 1.0)],
 [('고딩때야', 1.0)],
 [('개똑같아', 1.0), ('지금이랑', 1.0)],
 [('귀엽지', 1.0), ('얘그라', 1.0), ('이거어때', 1.0)],
 [('똑같다니', 1.0), ('야', 1.0)],
 [('귀여오', 1.0)],
 [('시부랭', 1.0)],
 [('이주희', 1.0), ('작가임', 1.0)],
 [('에바참치쌈싸서', 1.0), ('저얼굴이면', 1.0), ('지금도', 1.0), ('한입에털어넣야됨', 1.0)],
 [('털어넣으셈', 1.0)],
 [('근데', 1.0), ('똑같음', 1.0)],
 [('참치쌈싸서', 1.0), ('털어놓어야함', 1.0)],
 [('똑같다고', 1.0), ('레알', 1.0)],
 [('개충격', 1.0), ('시방', 1.0)],
 [('컴퍼스', 1.0)],
 [('잠깐만', 1.0)],
 [('레알똑같넼', 1.0)],
 [('시부랰',

In [51]:
# Author Topic Model 실행
NUM_TOPICS = 4

if not os.path.exists("kakao(ATM)_model"):
    model = AuthorTopicModel(corpus=corpus, 
                             id2word=dictionary.id2token,
                             num_topics=NUM_TOPICS, 
                             author2doc=author2doc, 
                             passes=10,
                             serialized=True,
                             serialization_path='serialized_model.mm') # passes: 학습의 반복횟수 (통상적으론 1000정도 많이 넣어줄수록 성능 good)
    # author2doc : 저자가 어떤 문서들을 썼는지에 대한 인덱스가 들어있는 dictionary
    # key:저자, value:index가 들어있는 리스트
    model.save("kakao(ATM)_model")
    
else:
    model = AuthorTopicModel.load("kakao(ATM)_model")

In [52]:
# 학습 결과 확인하기
model.show_topic(3, topn=20)
# 왜 의미없냐라고하면, 지금 우리가 다루는 데이터는 noise가 많은 데이터
# 토픽모델링에서 작동하기에는 굉장히 힘들어
# 문서의 개수는 많은데 단어가 짧지 -> 소셜 미디어의 단점

[('근데', 0.020257575241443723),
 ('야', 0.01664985607674788),
 ('진짜', 0.01269582066728958),
 ('나', 0.008903744398160406),
 ('저거', 0.007241612363562459),
 ('존나', 0.005750830621767807),
 ('다', 0.005399047101279087),
 ('너', 0.005270042038151068),
 ('우리', 0.004973752754808062),
 ('저', 0.004406656676373529),
 ('갑자기', 0.004245637255137831),
 ('나는', 0.004160489283577745),
 ('걍', 0.004155221036193371),
 ('개', 0.0038593066438886037),
 ('지금', 0.003840551003999961),
 ('그리고', 0.0032784088656344287),
 ('일단', 0.002718878618554346),
 ('아니', 0.0026420094073711484),
 ('그니까', 0.0024378255446064327),
 ('나도', 0.00241772357759156)]